In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('times.csv')
sentences = np.array(data.sentence)
tokens = np.array(data.token)
labels = np.array(data.label)
symbols_in_word  = 8

In [3]:
encoded_tokens = []
for i in range(len(tokens)):
    if type(tokens[i]) != str:
        tokens[i] = ''
symbol_map = dict()
inverted_symbol_map = dict()
symbol_counter = 1
for token in tokens:
    encoded_token = []
    for symbol in token:         
        if symbol_map.get(symbol) == None:
            symbol_map[symbol]=symbol_counter
            inverted_symbol_map[symbol_counter]=symbol
            symbol_counter+=1
        encoded_token.append(symbol_map.get(symbol))
    encoded_tokens.append(encoded_token)

In [4]:
def fitToSize(word, n):
    wordFit = word[:n].copy()
    if n > len(word):
        for i in range(n - len(word)):
            wordFit.append(0)   
    wordFit.reverse()
    return wordFit

def fitToSizeRev(word, n):
    wordFit = word.copy();
    wordFit.reverse()
    wordFit = wordFit[:n]
    if n > len(word):
        for i in range(n - len(word)):
            wordFit.append(0)  
    wordFit.reverse()
    return wordFit

In [5]:
encoded_token_rev = []
for i in range(len(encoded_tokens)):
    encoded_token_rev.append(fitToSizeRev(encoded_tokens[i],symbols_in_word))
    encoded_tokens[i] = fitToSize(encoded_tokens[i],symbols_in_word)

In [6]:
encoded_labels = []
for label in labels:
    ans = [5]
    if label == 'NONE':
        ans[0] = 0        
    elif label == 'D_ST': 
        ans[0] = 1   
    elif label == 'D_PT':
        ans[0] = 2
    elif label == 'D_EN':
        ans[0] = 3
    elif label == 'D_OR':
        ans[0] = 4   
    encoded_labels.append(ans)

In [7]:
batchesX = []
batchesY = []
batchesXrev = []
for i in range(sentences[-1]+1):
    batchesX.append([])
    batchesXrev.append([])
    batchesY.append([])
for i in range(len(encoded_tokens)):
    (batchesX[sentences[i]]).append(encoded_tokens[i])
    (batchesXrev[sentences[i]]).append(encoded_token_rev[i])
    (batchesY[sentences[i]]).append(encoded_labels[i])

In [8]:
def getTargetBatch(n):
    sizes = [2**i for i in range(2,11)]
    i = 0
    while n > sizes[i]:
        i+=1
    return i

In [9]:
TrainSize = int(len(batchesX)*0.8)

In [10]:
batchesXTrain = batchesX[:TrainSize]
batchesXrevTrain = batchesXrev[:TrainSize]
batchesYTrain = batchesY[:TrainSize]
batchesXTest = batchesX[TrainSize:]
batchesXrevTest = batchesXrev[TrainSize:]
batchesYTest = batchesY[TrainSize:]

In [11]:
def align(dataX, dataXrev, dataY): 
    if (len(dataX) == 0):
        return [], [], []
    maxl = np.max([np.shape(dataX[i])[0] for i in range(len(dataX))])
    batchSize = len(dataX)
    batchX = np.ones((batchSize, maxl, np.shape(dataX[0])[1]), np.float32)*147
    batchXrev = np.ones((batchSize, maxl, np.shape(dataX[0])[1]), np.float32)*147
    batchY = np.ones((batchSize, maxl,1), np.int32)*6   
    j = 0
    for i in range(len(dataX)):
        curLgth = np.shape(dataX[i])[0]
        np.copyto(batchX[j,0:curLgth,0:np.shape(dataX[0])[1]], dataX[i])   
        np.copyto(batchXrev[j,0:curLgth,0:np.shape(dataX[0])[1]], dataXrev[i])   
        np.copyto(batchY[j,0:curLgth],dataY[i])
        for k in range (curLgth, maxl):
               batchY[j][k] = [5]       
        j = j+1   
    return batchX, batchXrev, batchY 

In [12]:
def make_batches_per_size(BatchesX, BatchesXrev, BatchesY):
    sentensesPerSizeX = []
    sentensesPerSizeXrev = []
    sentensesPerSizeY = []
    for i in range(9):
        sentensesPerSizeX.append([])
        sentensesPerSizeXrev.append([])
        sentensesPerSizeY.append([])
    for i in range(len(BatchesX)):
        sentSize = np.shape(BatchesX[i])[0]
        sentensesPerSizeX[getTargetBatch(sentSize)].append(BatchesX[i])
        sentensesPerSizeXrev[getTargetBatch(sentSize)].append(BatchesXrev[i])
        sentensesPerSizeY[getTargetBatch(sentSize)].append(BatchesY[i])
    for i in range(len(sentensesPerSizeX)):
        sentensesPerSizeX[i], sentensesPerSizeXrev[i], sentensesPerSizeY[i] = align( sentensesPerSizeX[i], sentensesPerSizeXrev[i], sentensesPerSizeY[i])        
    return sentensesPerSizeX, sentensesPerSizeXrev, sentensesPerSizeY

In [13]:
SentXTrain,SentXTrainrev, SentYTrain = make_batches_per_size(batchesXTrain,batchesXrevTrain,batchesYTrain)
SentXTest,SentXTestrev, SentYTest = make_batches_per_size(batchesXTest,batchesXrevTest,batchesYTest)

In [14]:
from keras.utils.np_utils import to_categorical
def tocategorial(inp):
    caregorial = []
    for i in range(len(inp)):
        caregorial.append(to_categorical(inp[i],num_classes =6))
    return caregorial

Using Theano backend.
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 740M (CNMeM is enabled with initial size: 60.0% of memory, cuDNN 5110)


In [15]:
def decode_answer(answer):
    decoded_answer = []
    for i in range(len(answer)):
        for j in range(len(answer[i])):
            if np.argmax(answer[i][j]) == 0:
                decoded_answer.append('NONE')
            elif np.argmax(answer[i][j]) == 1:
                decoded_answer.append('D_ST')
            elif np.argmax(answer[i][j]) == 2:
                decoded_answer.append('D_PT')        
            elif np.argmax(answer[i][j]) == 3:
                decoded_answer.append('D_EN')   
            elif np.argmax(answer[i][j]) == 4:
                decoded_answer.append('D_OR') 
            elif np.argmax(answer[i][j]) == 5:
                decoded_answer.append('BLNK') 
    return decoded_answer;

In [16]:
def calculate_result(corpus, net):
    TP = 0
    FP = 0
    FN = 0
    counter = 0
    while counter < len(corpus):
        if corpus[counter] == 'NONE':
            if net[counter] != 'NONE':
                FP+=1
            counter+=1
        elif corpus[counter] == 'D_ST':
            i = 1
            while corpus[counter+i] != 'D_EN':
                i+=1
            i+=1
            flag = True
            for j in range(i):
                if net[counter+j] == 'NONE':
                    flag = False
            if flag == True:
                TP+=1
            else:
                FN+=1
            counter+=i
        elif corpus[counter] == 'D_OR':
            if net[counter] != 'NONE':
                TP+=1
            else:
                FN+=1
            counter+=1
        else:
            counter+=1
    print("TP: ", TP, "FP: ", FP, "FN: ", FN)
    if (TP == 0):
        TP+=1
    Precision = float(TP)/float(TP+FP)
    Recall = float(TP)/float(TP+FN)
    Fmeasure =  2.*(Precision*Recall)/(Precision+Recall)
    print('Precision', Precision)
    print('Recall', Recall)
    print('Fmeasure', 2.*(Precision*Recall)/(Precision+Recall))   
    return Fmeasure;

In [17]:
def calculate_accuracy(corpus, net):
    true_labels = 0
    false_labels = 0
    for i in range(len(corpus)):
        if corpus[i] != 'BLNK':
            if corpus[i] == net[i]:
                true_labels+=1
            else:
                false_labels+=1
    accuracy = float(true_labels)/float(true_labels+false_labels);
    print('Accuracy', accuracy)
    return accuracy

In [18]:
def calculate_statistics(Xdata, Xdatarev, Ydata):
    decoded_answer = []
    decoded_test = []
    for i in [i for i in range(2,11)]:
        sentPerBatch = int((512)/(2**i) * 40)
        k = 0
        for j in range(int(len(Xdata[i-2])/sentPerBatch)):                  
            answer = decode_answer( model.predict_on_batch([Xdata[i-2][j*sentPerBatch:(j+1)*sentPerBatch],
                                                          Xdatarev[i-2][j*sentPerBatch:(j+1)*sentPerBatch]]) )
            test = decode_answer(tocategorial( Ydata[i-2][j*sentPerBatch:(j+1)*sentPerBatch]) )        
            for opt in range(len(answer)):
                decoded_answer.append(answer[opt])
                decoded_test.append(test[opt])            
            k = j+1         
        if np.shape(Xdata[i-2])[0] == 0: 
            continue
        answer = decode_answer(model.predict_on_batch([Xdata[i-2][k*sentPerBatch:], Xdatarev[i-2][k*sentPerBatch:]]))
        test = decode_answer(tocategorial(Ydata[i-2][k*sentPerBatch:]))
        for opt in range(len(answer)):
            decoded_answer.append(answer[opt])
            decoded_test.append(test[opt]) 

    return calculate_result(decoded_test,decoded_answer), calculate_accuracy(decoded_test, decoded_answer)


In [19]:
def shuffle_butch(dataX, dataXrev, dataY):
    indexes = np.array(range(len(dataX)))
    np.random.shuffle(indexes)
    return dataX[indexes], dataXrev[indexes], dataY[indexes]

In [20]:
from keras.layers import Input
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense
from keras.models import Model
from keras.layers import Masking
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers.merge import concatenate

In [21]:
inputLayer1 = Input(dtype='int32', shape=(None,symbols_in_word,), name = 'inputLayer1')
embeddingLayer1 = TimeDistributed(Embedding(input_dim = 148, output_dim = 10, trainable=True, 
                                         name = 'embeddingLayer1'), name='TDEmbedding1')(inputLayer1)
embeddingLSTM1 = TimeDistributed(LSTM(50, return_sequences = False,name = 'embeddingLSTM1'), name='TDembeddinglstm1')(embeddingLayer1)

inputLayer2 = Input(dtype='int32', shape=(None,symbols_in_word,), name = 'inputLayer2')
embeddingLayer2 = TimeDistributed(Embedding(input_dim = 148, output_dim = 10, trainable=True, 
                                         name = 'embeddingLayer2'), name='TDEmbedding2')(inputLayer2)
embeddingLSTM2 = TimeDistributed(LSTM(50, return_sequences = False,name = 'embeddingLSTM2'), name='TDembeddinglstm2')(embeddingLayer2)
# mask = Masking(mask_value=147., name="Masking")(inputLayer)

mergeLayer = concatenate([embeddingLSTM1, embeddingLSTM2 ], name='mergeLayer')
dropuotLayer = Dropout(0.1, name='dropoutLayer')(mergeLayer)
lstmLayer = Bidirectional(LSTM(150, return_sequences = True, name = 'lstmLayer'), name='bidirectional2')(dropuotLayer)
outputLayer = TimeDistributed(Dense(6, activation='softmax',name='outputLayer'),
                              name='TDout')(lstmLayer)

model = Model(inputs=[inputLayer1, inputLayer2], outputs=[outputLayer])
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
inputLayer1 (InputLayer)         (None, None, 8)       0                                            
____________________________________________________________________________________________________
inputLayer2 (InputLayer)         (None, None, 8)       0                                            
____________________________________________________________________________________________________
TDEmbedding1 (TimeDistributed)   (None, None, 8, 10)   1480                                         
____________________________________________________________________________________________________
TDEmbedding2 (TimeDistributed)   (None, None, 8, 10)   1480                                         
___________________________________________________________________________________________

In [22]:
fmeasuresTest = []
fmeasuresTrain = []
accuracyTest = []
accuracyTrain = []

for epoch in range(200):
    print("epoch: ", epoch+1)
    external_indexes = np.array(range(2,11))
    np.random.shuffle(external_indexes)
    for i in external_indexes:
        sentPerBatch = int((512)/(2**i) * 40)
        k = 0
        batch_indexes = np.array(range(int(len(SentXTrain[i-2])/sentPerBatch)))
        np.random.shuffle(batch_indexes)
        for j in batch_indexes:
#             print(i,j)
            Xbatch, Xrevbatch, Ybatch = shuffle_butch(SentXTrain[i-2][j*sentPerBatch:(j+1)*sentPerBatch], 
                                                      SentXTrainrev[i-2][j*sentPerBatch:(j+1)*sentPerBatch],
                                 SentYTrain[i-2][j*sentPerBatch:(j+1)*sentPerBatch])
            model.train_on_batch([Xbatch,Xrevbatch],Ybatch)
            k += 1
#         print(i,k)
        Xbatch,Xrevbatch, Ybatch = shuffle_butch(SentXTrain[i-2][k*sentPerBatch:],
                                       SentXTrainrev[i-2][k*sentPerBatch:],
                                       SentYTrain[i-2][k*sentPerBatch:]) 
        model.train_on_batch([Xbatch,Xrevbatch],Ybatch)
    
    print("Train: ")
    ft, at = calculate_statistics(SentXTrain, SentXTrainrev, SentYTrain)
    fmeasuresTrain.append(ft)
    accuracyTrain.append(at)
    print("Test: ")
    ft, at = calculate_statistics(SentXTest,SentXTestrev, SentYTest )
    fmeasuresTest.append(ft)
    accuracyTest.append(at)

epoch:  1
Train: 
TP:  493 FP:  20796 FN:  6875
Precision 0.023157499177979237
Recall 0.06691096634093377
Fmeasure 0.034406951181212266
Accuracy 0.9220467361177482
Test: 
TP:  93 FP:  5510 FN:  1187
Precision 0.016598250936998037
Recall 0.07265625
Fmeasure 0.027023100392270807
Accuracy 0.9291931504462115
epoch:  2
Train: 
TP:  3 FP:  16 FN:  7365
Precision 0.15789473684210525
Recall 0.00040716612377850165
Fmeasure 0.000812237714904562
Accuracy 0.9716978877950875
Test: 
TP:  0 FP:  0 FN:  1280
Precision 1.0
Recall 0.00078064012490242
Fmeasure 0.0015600624024961
Accuracy 0.9797301635344725
epoch:  3
Train: 
TP:  17 FP:  56 FN:  7351
Precision 0.2328767123287671
Recall 0.0023072747014115094
Fmeasure 0.004569278322806075
Accuracy 0.9716023129121667
Test: 
TP:  0 FP:  0 FN:  1280
Precision 1.0
Recall 0.00078064012490242
Fmeasure 0.0015600624024961
Accuracy 0.9797301635344725
epoch:  4
Train: 
TP:  17 FP:  60 FN:  7351
Precision 0.22077922077922077
Recall 0.0023072747014115094
Fmeasure 0.004

In [23]:
np.max(fmeasuresTest)

0.71203776553894582

In [24]:
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: TkAgg


In [25]:
plt.plot(range(len(fmeasuresTest)),fmeasuresTest)
plt.plot(range(len(fmeasuresTrain)),fmeasuresTrain)
plt.grid(True)
plt.show()

In [26]:
DATA = []
INDEX = []
for i in range(len(fmeasuresTest)):
    DATA.append([fmeasuresTest[i],fmeasuresTrain[i],accuracyTest[i],accuracyTrain[i]])
    INDEX.append(i)
    
df = pd.DataFrame(data= DATA, index=INDEX)                        
df.to_csv('lstm8_2.csv')

In [27]:
print kek

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-27-aa1d76923acd>, line 1)

In [ ]:
butches_count_train = int(len(batchesX)*0.8)
trainX =  batchesX[:butches_count_train]
trainY = batchesY[:butches_count_train]
fmTrain = []
fmTest = []
print 
maxFmeasure = 0
for epoch in range(10):
    shuffle_indexes = np.array(range(butches_count_train))
    np.random.shuffle(shuffle_indexes)
#     trainX = trainX[shuffle_indexes]
#     trainY = trainY[shuffle_indexes]
    print("epoch: ", epoch+1)   
   
    for i in range(350):
        print("batch: ", i+1)        
        batchX, batchY, batchMask = createBatch(shuffle_indexes[50*i:50*(i+1)], trainX, trainY)
#         print("Xshape", batchX.shape)
#         print("Yshape", batchY.shape)  
        model.train_on_batch(batchX, batchY) 
    print("batch: ", 175)
    batchX, batchY, batchMask = createBatch(shuffle_indexes[50*350:], trainX, trainY)
    print("Xshape", batchX.shape)
    print("Yshape", batchY.shape) 
    model.train_on_batch(batchX, batchY)  
#          for i in range(len(batchesX[:butches_count_train])):    
#         if (i % 1000 == 0):
#             print("batch: ", i)
#         model.train_on_batch(np.array([trainX[i]]), np.array([trainY[i]]))
   
#     print("Result on Train: ")
#     answer = []
#     for i in range(len(batchesX[:butches_count_train])):
#         answer.append(model.predict_on_batch(np.array([batchesX[i]])))   
#     decoded_answer = decode_answer(answer)
#     fmTrain.append(calculate_result(labels[:len(decoded_answer)],decoded_answer)) 
    
        
#     print("Result on Test: ")    
#     answer = []
#     for i in range(len(batchesX[butches_count_train:])):
#         answer.append(model.predict_on_batch(np.array([batchesX[butches_count_train+i]])))   
#     decoded_answer = decode_answer(answer)
#     fmTest.append(calculate_result(labels[-len(decoded_answer):],decoded_answer))
 

In [ ]:
print(butches_count_train) 

In [ ]:
print("Result on Train: ")
answer = []
for i in range(len(batchesX[:butches_count_train])):
    answer.append(model.predict_on_batch(np.array([batchesX[i]])))   
decoded_answer = decode_answer(answer)
fmTrain.append(calculate_result(labels[:len(decoded_answer)],decoded_answer))

In [ ]:
print("Result on Test: ")    
answer = []
for i in range(len(batchesX[butches_count_train:])):
    answer.append(model.predict_on_batch(np.array([batchesX[butches_count_train+i]])))   
decoded_answer = decode_answer(answer)
fmTest.append(calculate_result(labels[-len(decoded_answer):],decoded_answer))

In [ ]:
aer = np.sort([np.shape(batchesX[i])[0] for i in range(len(batchesX))])
aer[-100:]

In [ ]:
# answer = []
# for i in range(len(batchesX[butches_count_train:])):
#     answer.append(model.predict_on_batch(np.array([batchesX[butches_count_train+i]])))
answer = []
for i in range(len(batchesX[:butches_count_train])):
    answer.append(model.predict_on_batch(np.array([batchesX[i]])))

In [ ]:
# df = pd.DataFrame(data= decoded_answer, index = labels[-len(decoded_answer):])                        
# df.to_csv('answer.csv')
df = pd.DataFrame(data= decoded_answer, index = decoded_test)                        
df.to_csv('answer_train.csv')

In [ ]:
from keras.utils.np_utils import to_categorical
int_labels = [1,3,2,1,0,0,2,1]
categorical_labels = to_categorical(int_labels, num_classes=None)
categorical_labels